<a href="https://colab.research.google.com/github/su92-msaiw-f25-016-spec/lessons-learned/blob/Machine_Learning_Project/Priceprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

**LOAD DATA**

In [7]:
data = pd.read_csv("price_classification_dataset.csv")